## 3584번: 가장 가까운 공통 조상

In [13]:
import sys
from collections import deque
#input = sys.stdin.readline

def LCA(a, b):
    if depth[a] > depth[b]:
        a, b = b, a
    while depth[a] != depth[b]:
        b = before[b]
    while a != b:
        a = before[a]
        b = before[b]
    return a

def find_root(graph):
    count = [0] * (N+1)
    count[0] = 1
    for i in graph:
        for j in i:
            count[j] = 1
    return count.index(0)

T = int(input())
for t in range(T):
    N = int(input())
    graph = [[] for i in range(N+1)]
    for _ in range(N-1):
        A, B = map(int, input().split())
        graph[A].append(B)

    depth = [0] * (N+1)
    before = [0] * (N+1)
    root = find_root(graph)
    queue = deque()
    queue.append(root)
    while queue:
        select = queue.popleft()
        for n in graph[select]:
            if depth[n] == 0:
                queue.append(n)
                depth[n] = depth[select] + 1
                before[n] = select
    A, B = map(int, input().split())
    print(LCA(A, B))

2
16
1 14
8 5
10 16
5 9
4 6
8 4
4 10
1 13
6 15
10 11
6 7
10 2
16 3
8 1
16 12
16 7
4
5
2 3
3 4
3 1
1 5
3 5
3


## 17435번: 합성함수와 쿼리

In [23]:
import sys
#input = sys.stdin.readline
m = int(input())
f = [[0] * (m+1) for i in range(20)]
f[0] = [0] + list(map(int, input().split()))

for i in range(1, 20):
    for j in range(1, m+1):
        f[i][j] = f[i-1][f[i-1][j]]
        
q =  int(input())
for _ in range(q):
    n, x = map(int, input().split())
    for i in range(18, -1, -1):
        if n >= 2**i:
            n -= 2**i
            x = f[i][x]
    print(x)

5
3 3 5 4 3
5
1 1
3
2 1
5
11 3
5
1000 4
4
5 1
3


## 11438번: LCA 2

In [11]:
import sys
#input = sys.stdin.readline
sys.setrecursionlimit(100000) #재귀 깊이 미설정시 RecursionError 발생

def DFS(x, d):
    visited[x] = 1
    depth[x] = d
    for n in graph[x]:
        if visited[n] != 1:
            parents[n][0] = x
            DFS(n, d+1)

def sparse_table():
    for j in range(1, 21):
        for i in range(1, N+1):
            parents[i][j] = parents[parents[i][j-1]][j-1]
            
def LCA(x, y):
    if depth[x]>depth[y]:
        x, y = y, x
    for i in range(20, -1, -1):
        if(depth[y]-depth[x]>=(1<<i)):
            y=parents[y][i]     
    #print(x, y)
    if x==y:
        return x
    for i in range(20, -1, -1):
        if parents[x][i]!=parents[y][i]:
            x = parents[x][i]
            y = parents[y][i]
    return parents[x][0]

N = int(input())
graph = [[] for i in range(N+1)]
for _ in range(N-1):
    A, B = map(int, input().split())
    graph[A].append(B)
    graph[B].append(A)
    
depth = [0] * (N+1)
root = 1
visited = [0] * (N+1)
parents = [[0]*21 for i in range(N+1)]
DFS(root, 0)

sparse_table()
M = int(input())
for _ in range(M):
    A, B = map(int, input().split())
    print(LCA(A, B))

15
1 2
1 3
2 4
3 7
6 2
3 8
4 9
2 5
5 11
7 13
10 4
11 15
12 5
14 7
6
6 11
2
10 9
4
2 6
2
7 6
1
8 13
3
8 15
1


## 3176번: 도로 네트워크

In [75]:
import sys
from math import log2
from collections import deque
#input = sys.stdin.readline

N = int(input())
H = int(log2(N))+1
graph = [[] for i in range(N+1)]
for _ in range(N-1):
    A, B, C = map(int, input().split())
    graph[A].append((B, C))
    graph[B].append((A, C))
    
queue = deque((1, 1))
depth = [0] * (N+1) #트리 깊이
depth[1] = 1
dp = [[[0, 0, 0] for _ in range(H)] for _ in range(N+1)]

while queue:
    s = queue.popleft()
    for n, w in graph[s]:
        if not depth[n]:
            queue.append(n)
            depth[n] = depth[s]+1
            dp[n][0] = [s, w, w]
            
#sparse table 작성
for j in range(1, H):
        for i in range(1, N+1):
            dp[i][j][0] = dp[dp[i][j-1][0]][j-1][0]
            dp[i][j][1] = min(dp[i][j-1][1], dp[dp[i][j-1][0]][j-1][1])
            dp[i][j][2] = max(dp[i][j-1][2], dp[dp[i][j-1][0]][j-1][2])
            
for _ in range(int(input())):
    D, E = map(int, input().split())
    max_len = 0
    min_len = sys.maxsize
    
    #깊이가 같을때까지 올라가면서 최대값, 최소값 저장
    if depth[D] > depth[E]:
        D, E = E, D
    for i in range(H, -1, -1):
        if depth[E]-depth[D]>=(1<<i):
            min_len = min(min_len, dp[E][i][1])
            max_len = max(max_len, dp[E][i][2])
            E = dp[E][i][0]
    if D == E:
        print(min_len, max_len)
        continue

    #공통 부모 찾으면서 최대값, 최소값 저장
    for i in range(H-1, -1, -1):
        if dp[D][i][0]!=dp[E][i][0]:
            min_len = min(min_len, dp[D][i][1], dp[E][i][1])
            max_len = max(max_len, dp[D][i][2], dp[E][i][2])
            D, E = dp[D][i][0], dp[E][i][0]
            
    min_len = min(dp[D][0][1],dp[E][0][1], min_len)
    max_len = max(dp[D][0][2],dp[E][0][2], max_len)

    print(min_len, max_len)

9
1 2 2
2 3 1
3 4 5
2 7 4
1 5 3
5 6 1
5 9 2
1 8 3
5
6 9
1 2
7 8
2 4
9 4
1 5
1 2
2 2
7 3
1 4
